# IFT6269 - Homework 3 - EM and Gaussian mixtures
**Due:**  Tuesday, November 8, 2022

#### Name: Tejas Vaidhya
#### Student ID: uv56320
#### Collaborators: 



## Introduction

The file `EMGaussian.train` contains samples of data $\{x_n\}_{n=1}^N$ where $x_n \in \mathbb{R}^2$, with one datapoint per row. The goal of this exercise is to implement the K-Means and EM algorithms using $K=4$ components/clusters. 

### Tasks
0.   Get your own copy of this file via "File > Save a copy in Drive...",
1.   Fill your personal information and collaborators at the top of this assignment, and rename the notebook accordingly, e.g., `hw3_thomasBayes.ipynb`
2.   Read the instructions provided on each section and cell carefully,
4.   Implement the requested algorithms in section **Playground**
5.   In section **Model Comparison**, simply execute the cells (without changing the code) and type your answers to the questions in the provided text cells.
    
**Important**: You are allowed to collaborate with other students in both the math and coding parts of this assignment. However, the answers provided here must reflect your individual work. For that reason, you are not allowed to share this notebook, except for your submission to the TA for grading. **Don't forget to pin and save the version of the notebook you want to be graded on!**

In [1]:
!wget http://www.iro.umontreal.ca/~slacoste/teaching/ift6269/A21/notes/hwk3data.zip
!unzip -o hwk3data.zip

import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

X_train = np.loadtxt("/content/hwk3data/EMGaussian.train")
X_test = np.loadtxt("/content/hwk3data/EMGaussian.test")

--2022-10-24 02:12:40--  http://www.iro.umontreal.ca/~slacoste/teaching/ift6269/A21/notes/hwk3data.zip
Resolving www.iro.umontreal.ca (www.iro.umontreal.ca)... 132.204.26.36
Connecting to www.iro.umontreal.ca (www.iro.umontreal.ca)|132.204.26.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7269 (7.1K) [application/zip]
Saving to: ‘hwk3data.zip’

hwk3data.zip        100%[===================>]   7.10K  --.-KB/s    in 0s      

2022-10-24 02:12:41 (647 MB/s) - ‘hwk3data.zip’ saved [7269/7269]

Archive:  hwk3data.zip
  inflating: hwk3data/EMGaussian.test  
  inflating: hwk3data/EMGaussian.train  


## Playground

You are allowed to add as many cells and functions as you wish in this section, but not allowed to change the signature (name and inputs) of the functions!

In [ ]:
#@title Plotting code _(execute ▸ , but do not modify)_
# ---------------------------------------------------------------------------
#                       Code for plotting the results 
#                      ! DO NOT MODIFY THESE FUNCTIONS !
# ---------------------------------------------------------------------------

from matplotlib.patches import Ellipse
def plot_ellipse(ax, mu, sigma, alpha=1, color="k"):
    evals, evecs = np.linalg.eigh(sigma)
    x, y = evecs[:, 0]
    theta = np.degrees(np.arctan2(y, x))
    for factor in [1.5, 3]:
        w, h = factor * np.sqrt(evals)
        ellipse = Ellipse(mu, w, h, theta, facecolor='none', edgecolor=color) 
        ellipse.set_alpha(alpha)
        ax.add_artist(ellipse) 
        
def show_kmeans(X_train, X_test, KM_centroids, KM_predictor):
    
    shapes = ['o', '*', 'v', '+']
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#9467bd', '#8c564b',
              '#e377c2', '#7f7f7f', '#bcbd22']
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    
    for (ax, data, title) in [(ax1, X_train, 'Training Set'), (ax2, X_test, 'Test Set')]:
        pred_labels, obj = KM_predictor(data)
        print("K-Means Objective on " + title + ": ", obj)
        cs = [colors[int(_) % len(colors)] for _ in pred_labels]
        ax.scatter(data[:, 0], data[:, 1], alpha=0.5, c=cs)
        ax.scatter(KM_centroids[:, 0], KM_centroids[:, 1], marker='o', c='#d62728')
        ax.set_title(title)
        ax.set_xlim(-12, 12), ax.set_ylim(-12, 12)
        ax.set_aspect('equal')    
        
    plt.show()
    print('\n')
    
    
def show_mog(X_train, X_test, MoG_pi, MoG_centroids, MoG_sigmas, MoG_predictor):
    
    shapes = ['o', '*', 'v', '+']  
    colors = [[31, 119, 180], [255, 127, 14], [44, 160, 44], [148, 103, 189],
             [140, 86, 75], [227, 119, 194], [127, 127, 127], [188, 189, 34]]
    
    max_pi = np.max(MoG_pi)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    
    for (ax, data, title) in [(ax1, X_train, 'Training Set'), (ax2, X_test, 'Test Set')]:
        pred_labels, norm_llike = MoG_predictor(data)
        print("MoG Normalized Log-Likelihood " + title + ": ", norm_llike)
        cs = [colors[int(_) % len(colors)] + [0.5*255*MoG_pi[_]/max_pi] for _ in pred_labels]
        ax.scatter(data[:, 0], data[:, 1], c=np.array(cs)/255.)
        ax.scatter(MoG_centroids[:, 0], MoG_centroids[:, 1], marker='o', c='#d62728')
        ax.set_title(title)
        ax.set_xlim(-12, 12), ax.set_ylim(-12, 12)
        ax.set_aspect('equal')    
        
        for _ in range(MoG_pi.shape[0]):
            plot_ellipse(ax, MoG_centroids[_, :], MoG_sigmas[_, :,:], alpha=MoG_pi[_]/max_pi, color='k')
    
    plt.show()
    print('\n')

In [ ]:
def KMeans(X, K=1):
    """
    Estimates the parameters of a K-means model using training data X
        
        Inputs:
            X: [nx2] matrix of inputs
            K: [int] number of clusters to use
            
        Returns:
            centroids: [Kx2] matrix of estimated centroid locations
            KMeans_predictor: function taking a matrix inX and returning the
                predicted cluster number (starting from 0 to K-1) for each row 
                of inX; and the normalized KMeans loss (i.e., divided by the
                number of rows of inX).
                 
    """
    
    # TODO  
    centroids = np.random.randn(K, X.shape[1])
    old_obj = 0
    new_obj = 0
    while True:
        #E-step
        dist_matrix = np.sum((X[:, np.newaxis, :] - centroids[np.newaxis, :, :])**2, axis=2)
        pred_labels = np.argmin(dist_matrix, axis=1)
        #M-step
        # use only matrix for get new centroids
        centroids = ((X[:, np.newaxis, :] * (pred_labels[:, np.newaxis, np.newaxis] == np.arange(K)[np.newaxis, np.newaxis, :])).sum(axis=0) / (pred_labels[:, np.newaxis, np.newaxis] == np.arange(K)[np.newaxis, np.newaxis, :]).sum(axis=0))

        #objective
        new_obj = KMeans_objective(X, centroids, pred_labels)
        if np.abs(new_obj - old_obj) < 1e-6:
            break
        old_obj = new_obj
     
    def KMeans_objective(X, centroids, pred_labels):
        dist_matrix = np.sum((X[:, np.newaxis, :] - centroids[np.newaxis, :, :])**2, axis=2)
        return np.sum(dist_matrix[np.arange(X.shape[0]), pred_labels]) / X.shape[0]

    def KMeans_predictor(inX):
        """
        Use parameters from above to predict cluster for each row of inX
        
            Inputs:
                inX: [mx2] matrix of inputs
                
            Returns:
                pred_labels: [m] array of predicted cluster labels
                norm_loss: [float] K-Means loss on inX, i.e. the *squared*
                    distance between each point and its associated cluster,
                    divided by the number of rows of inX.
        """

        # TODO
        m = inX.shape[0]
        pred_labels = np.random.randint(0, K, m)  # This must be a vector of integers
        norm_loss = 0. / m
        dist = np.zeros((inX.shape[0], K))
        for i in range(K):
            dist[:, i] = np.linalg.norm(inX - centroids[i], axis=1)
        pred_labels = np.argmin(dist, axis=1)
        norm_loss = KMeans_objective(inX, centroids, pred_labels)
        return pred_labels, norm_loss
        
    return centroids, KMeans_predictor

def GaussianMixture(X, K=1, use_full_cov=True):
    """
    Estimates the parameters of a Gaussian mixture model using training data X
    
    **Important:** The locations of the centroids must be initialized using your 
    K-Means code! With this information, initialize the proportions and variances
    accordingly.
    
        Inputs:
            X: [nx2] matrix of inputs
            K: [int] number of mixture components to use
            use_full_cov: [bool] if True, estimate a full covariance for each 
                mixture component. Else, we use a scaled identity for each 
                component (in this case each component might have a 
                different scaling of the identity: Sigma_i = sigma_i * I).
            
        Returns:
            pi: [K] vector of proportion of each class
            centroids: [Kx2] matrix of estimated centroid locations
            sigmas: [Kx2x2] tensor of estimated covariance matrices
            MoG_predictor: function taking a matrix inX and returning the
                predicted cluster number (starting from 0 to K-1) for each row 
                of inX; and the normalized log-likelihood (i.e., ln(p(inX)) divided by the
                number of rows of inX).
    """
    
    # Initialize centroids using KMeans
    centroids, _ = KMeans(X, K)
    
    # TODO

    # This is just boiler-plate code. Change this to do the right computations
    # but **keep the same names and shape for these variables** 
    pi = np.ones(K) / K
    old_obj = 0
    new_obj = 0
    
     # TODO
    if use_full_cov:
        # We use a full covariance for each class
        sigmas = np.concatenate(K * [np.eye(2)[np.newaxis, ...]], axis=0)
    else:
        # We use one scaled identity for each class
        sigmak2 = np.ones(K)  # These are the sigma^{2}_{k} for k = 1, ..., K
        sigmas = np.concatenate([sigmak2[i] * np.eye(2)[np.newaxis, ...] for i in range(K)], axis=0)
    
    while True:
        #E-step
        tau = np.zeros((X.shape[0], K))
        for i in range(K):
            tau[:, i] = pi[i] * np.exp(-0.5 * np.sum(np.matmul((X - centroids[i]), np.linalg.inv(sigmas[i])) * (X - centroids[i]), axis=1)) / np.sqrt(np.linalg.det(sigmas[i]))
        tau = tau / np.sum(tau, axis=1, keepdims=True)
        
        #M-step
        for i in range(K):
            pi[i] = np.sum(tau[:, i]) / X.shape[0]
            centroids[i] = np.sum(tau[:, i][:, np.newaxis] * X, axis=0) / np.sum(tau[:, i])
            sigmas[i] = np.sum(tau[:, i][:, np.newaxis, np.newaxis] * np.matmul((X - centroids[i])[:, :, np.newaxis], (X - centroids[i])[:, np.newaxis, :]), axis=0) / np.sum(tau[:, i])
        
        def GaussianMixture_objective(X, pi, centroids, sigmas):
            obj = 0
            for i in range(K):
                obj += np.sum(pi[i] * np.exp(-0.5 * np.sum(np.matmul((X - centroids[i]), np.linalg.inv(sigmas[i])) * (X - centroids[i]), axis=1)) / np.sqrt(np.linalg.det(sigmas[i])))
            return obj / X.shape[0]
        
        #objective
        new_obj = GaussianMixture_objective(X, pi, centroids, sigmas, tau)
        if np.abs(new_obj - old_obj) < 1e-6:
            break
        old_obj = new_obj


        
    def MoG_predictor(inX):
        """
        # Use parameters from above to predict cluster for each row of inX
        
            Inputs:
                inX: [mx2] matrix of inputs
                
            Returns:
                pred_labels: [m] array of predicted cluster labels
                norm_loglike: [float] the log-likelihood of inX, i.e. ln(p(inX)),
                    divided by the number of rows of inX.
        """
        # TODO
        m = inX.shape[0]
        pred_labels = np.random.randint(0, K, m)  # This must be a vector of integers
        norm_loglike = 0. / m
        tau = np.zeros((inX.shape[0], K))
        for i in range(K):
            tau[:, i] = pi[i] * np.exp(-0.5 * np.sum(np.matmul((inX - centroids[i]), np.linalg.inv(sigmas[i])) * (inX - centroids[i]), axis=1)) / np.sqrt(np.linalg.det(sigmas[i]))
        tau = tau / np.sum(tau, axis=1, keepdims=True)
        pred_labels = np.argmax(tau, axis=1)
        # dont use MoG_objective
        norm_loglike = np.sum(np.log(np.sum(tau, axis=1))) / inX.shape[0]
        return pred_labels, norm_loglike
    
    return pi, centroids, sigmas, MoG_predictor    

: 

## Model Comparison

In this section **DO NOT** change the code in any of the cells. Simply answer the questions in the corresponding text cells after having executed your implementation. If you have respected the signature of the functions above in terms of inputs and outputs, your code should run. 

### K-Means


In [ ]:
for run in range(3):
    KM_centroids, KM_predictor = KMeans(X_train, K=4)
    show_kmeans(X_train, X_test, KM_centroids, KM_predictor)

We have trained your implementation of the K-Means algorithm using `X_train` for 3 different initializations. 

**Question:** Briefly compare the results above in terms of the location of the centers and the K-means training objective (at convergence) across runs with different initializations. What conclusions can you draw from this? 

**Answer:**

### EM for Gaussian mixture models 

In [ ]:
print('Scaled identity covariance matrices')
MoG1 = GaussianMixture(X_train, K=4, use_full_cov=False)
show_mog(X_train, X_test, MoG1[0], MoG1[1], MoG1[2], MoG1[3])

print('\nFull covariance matrices')
MoG2 = GaussianMixture(X_train, K=4, use_full_cov=True)
show_mog(X_train, X_test, MoG2[0], MoG2[1], MoG2[2], MoG2[3])

We have trained your implementation of the EM algorithm for a Gaussian mixture model using `X_train`. The plots show the behavior on the training and testing (left and right) sets when using scaled diagonal or full covariance matrices (top and bottom), respectively. 

**Question:** Briefly compare the results above in terms of the location of the centers and the normalized log-likelihood between the training and test set. What conclusions can you draw from this? 

**Answer:**

### Bonus
**Question:** From your observations in the K-Means and EM sections, is there any relation one can stablish between these two methods? If so, how do the plots above reflect this?  

**Answer:** 